## R - Projections and Vector Maps


In this notebook, we will see how to draw vector maps from different sources. 

In [ ]:
# Useful libraries to visualize maps 
library(ggplot2)
library(sp)
library(maps)
library(maptools)
library(mapproj)
library(mapdata)

Let's start with plain R to draw flights in between cities. 
Plain R can be sometimes shorter to code. 
In this example, our data set has two files; one file contains the flight information for airlines between airports, 
the other file contains the geo coordinates of the airports. 
First, we will go through the flight data and find out the coordinates of the originating and destination airports and visualize the number of flights. 

In [ ]:
# airport codes and coordinates 
airports <- read.csv("/dsa/data/all_datasets/spatial/airports.csv", as.is=TRUE, header=TRUE)
# flight destinations and counts 
flights <- read.csv("/dsa/data/all_datasets/spatial/flights.csv", as.is=TRUE, header=TRUE)

airports$lat <- as.numeric(airports$lat)
airports$long <- as.numeric(airports$long)

head(airports)
head(flights)

# Draw world map in plain R 
map("world", col="#f2f2f2", fill=TRUE, bg="white", lwd=0.05, xlim=c(-172, -57), ylim=c(12, 72))

# This is needed to compute the great circle between two locations on earth 
library(geosphere)

# Display only American Airlines 
fsub <- flights[flights$airline == "AA",]

# go through each flight and find the coordinates of originating and destination airport 
for (j in 1:length(fsub$airline)) {
    air1 <- airports[airports$iata == fsub[j,]$airport1,]
    air2 <- airports[airports$iata == fsub[j,]$airport2,]

    # compute the great circle and create a path of 100 points between endpoints 
    inter <- data.frame(gcIntermediate(c(air1[1,]$long, air1[1,]$lat), c(air2[1,]$long, air2[1,]$lat), n=100, addStartEnd=TRUE))
    
    # draw the arc for this flight; line thickness encodes number of flights 
    lines(inter, col="black", lwd=2*fsub[j,]$cnt/max(fsub$cnt))
}

We can also read the vector map data from a **shapefile**. 
A shapefile contains geospatial vector data that define geometric shapes of a map and the related attributes. Shapefiles can be obtained from online resources such as [US Census Bureau](https://www.census.gov/geo/maps-data/data/tiger-cart-boundary.html). 
We can read the shapefiles and convert them to data frames that ggplot can display. 
But they can be very large and take some time to process.

In [ ]:
# Load the shapefile and convert to a data frame
us_shp <- readShapePoly("/dsa/data/all_datasets/spatial/cb_2015_us_state_500k.shp")
us_map <- fortify(us_shp)
head(us_map)

ggplot(us_map, aes(x = long, y = lat, group=group)) + 
geom_path() + 
coord_map("mercator") + 
xlim(c(-172, -57)) + ylim(c(12, 72)) 


**Note:** In the plot above, the size of Alaska is abnormally large relative to the rest of the US.
This is a visualization effect of casting the coordinates from the ellipsoidal surface into a 2-D plane.

The plot below changes coordinate system from planar (mercator) to polyconic.

In [ ]:
ggplot(us_map, aes(x = long, y = lat, group=group)) + 
geom_path() + 
coord_map("polyconic") + 
xlim(c(-172, -57)) + ylim(c(12, 72)) 

Let's see how to get the vector data using map_data() function of maps library, which will be faster. 

In [ ]:
world <- map_data("world")
w <- ggplot()
w <- w + geom_map(data=world, map=world, aes(long, lat, map_id=region), color="black", fill="#d6bf86", size=0.1)
w <- w + coord_equal() 
w <- w + theme_void()
w

In [ ]:
# We can similarly get the US state data 
us <- map_data("state")

gg <- ggplot() + geom_map(data=us, map=us, aes(long, lat, map_id=region), color="#222222", fill=NA, size=0.15)
# this is a good projection for US
gg <- gg + coord_map("polyconic")
gg <- gg + theme_void() 
gg 

In [ ]:
gg <- ggplot() + geom_map(data=us, map=us, aes(long, lat, map_id=region), color="white", fill="lightblue", size=0.15)
#This is better projection
gg <- gg + coord_map("albers", lat0=30, lat1=40)
gg <- gg + theme_void()
gg

### This is how we can get data for different levels of detail and use different layers to overlay them. 

In [ ]:


state <- map_data("state")
county <- map_data("county")
usa <- map_data("usa")

gg <- ggplot()
gg <- gg + geom_map(data=county, map=county,
                    aes(long, lat, map_id=region),
                    color="grey", fill=NA, size=0.1)
gg <- gg + geom_map(data=state, map=state,
                    aes(long, lat, map_id=region),
                    color="red", fill=NA, size=0.3)
gg <- gg + geom_map(data=usa, map=usa,
                    aes(long, lat, map_id=region),
                    color="blue", fill=NA, size=0.6)
gg <- gg + coord_map("albers", lat0=30, lat1=40)
gg <- gg + theme_void()
gg

If you review the code that produced the above plot, we added geospatial layers in this order:
 1. Counties in `color="grey"` and `size=0.1`
 1. States in `color="red"` and `size=0.3`
 1. Country in `color="blue"` and `size=0.6`
 
This greats the visual effects of increasing composition that is tied to line thickness and size.

#### Looking back at the County data.

In [ ]:
head(county)

In [ ]:
#Let's pick Missouri
mo <- county[which(county$region=="missouri"),]

# same thing can be done like this:
mo2 <- map_data("county", "missouri")

# Let's select the Boone county from MO;  there are Boone counties in other states, too. 
boone <- county[which(county$subregion=="boone" & county$region=="missouri"),]


In [ ]:
gg <- ggplot()
gg <- gg + geom_map(data=mo, map=mo,
                    aes(x=long, y=lat, map_id=region),
                    color="black", fill=NA, size=0.1)
gg <- gg + geom_map(data=boone, map=boone,
                    aes(x=long, y=lat, map_id=region),
                    color="red", fill=NA, size=0.4)
gg <- gg + coord_map("polyconic")

gg <- gg + theme_void()
gg
head(mo)

# <span style="background:yellow">Your Turn</span>

Choose another state and county that is not Boone County, Missouri; such your current home location or birth place.
Plot the county within the appropriate state, without plotting the vectors of the other counties in that state.

For an **extra challege**, plot a rendering where that one county is rendered as a hole cute from the state.
Exmple: ![Missouri_without_Boone image](../images/Missouri_without_Boone.png)

In [ ]:
# Add your code below this comment
# --------------------------------






## Overlaying additional data

In [ ]:
# Let's get coordinates of some of the cities in MO and display them. 
head(us.cities)
mo_cities <- us.cities[which(us.cities$country.etc=="MO"),]
mo_cities

In [ ]:
# This is a bubble map 
gg + geom_point(data=mo_cities, aes(x=long, y=lat, size=pop, color=factor(capital))) +
scale_color_manual(values=c("blue","red"))

Now, let's finish with a similar plot like in the beginning, but this time using ggplot and dplyr for smarter data manipulation.

In [ ]:
# pick only busy routes 
flights <- flights[which(flights$cnt> 300),]

# get airport locations
airport_locs <- airports[, c("iata","long", "lat")]

library(dplyr)
# Link airport lat long to origin and destination
OD <- left_join(flights, airport_locs, by=c("airport1"="iata"))
OD <- left_join(OD, airport_locs, by=c("airport2"="iata"))
head(OD)

# Now, create curves on the map with a fixed curvature - no great circle computation 
ggplot() + 
#geom_polygon(data=world, aes(long, lat, group=group), fill="#e6ef86", color="black", size=0.1) +
geom_map(data=world,map=world, aes(long, lat, map_id=region), fill="#e6ef86", color="black", size=0.1) +

geom_curve(data=OD, aes(x=long.x, y=lat.x, xend=long.y, yend=lat.y, color=cnt), size=0.1,
                 curvature=-0.2, arrow=arrow(length=unit(0.01, "npc"))) +
    
scale_colour_distiller(palette="Blues", guide="none") +
coord_equal() +
xlim(c(-172,-57)) + ylim(c(12,72)) + 
theme_void()

# Save your notebook, then File > Close and Halt